In [3]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

<h2>Scraping the webpage

In [7]:
url='https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N'
page = requests.get(url)
doc= lh.fromstring(page.content)
tr_element = doc.xpath('//tr')

Checking the data downlaoded

In [75]:
[len(T) for T in tr_element[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [16]:
tr_element=doc.xpath('//tr')
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_element[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [27]:
for j in range(1,len(tr_element)):
    T=tr_element[j]
    if len(T)!=3:
        break
    i=0  
    for t in T.iterchildren():
        data=t.text_content()
        col[i][1].append(data)
        i+=1
            

In [30]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [74]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
Postcode          287 non-null object
Borough           287 non-null object
Neighbourhood
    287 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


<h2>Removing not assigned</h2>

In [73]:
df_toronto=df[df.Borough!='Not assigned'].reset_index(drop=True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M6A,North York,Lawrence Heights\n
4,M6A,North York,Lawrence Manor\n


<h2>not assigned neighbourhood</h2>

In [62]:
for index, row in df_toronto.iterrows():
    if(row["Neighbourhood\n"]=="Not assigned"):
        row["Neighbourhood"] = row["Borough"]

In [72]:
for index, row in df_toronto.iterrows():
    row["Neighbourhood\n"]=row["Neighbourhood\n"].strip()

In [71]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<h1> Group neighborhoods</h1>

In [66]:
df_toronto_grouped = df_toronto.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_toronto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [70]:
df_toronto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [68]:
column_names = ["Postcode", "Borough"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_toronto_grouped[df_toronto_grouped["Postcode"]==postcode], ignore_index=True)
    
test_df

,Borough,Neighbourhood,Postcode
0,Downtown Toronto,Central Bay Street,M5G
1,North York,Hillcrest Village,M2H
2,East York,"Woodbine Gardens, Parkview Hill",M4B
3,Scarborough,Scarborough Village,M1J
4,East York,Leaside,M4G
5,East Toronto,Studio District,M4M
6,Scarborough,"Maryvale, Wexford",M1R
7,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",M9V
8,North York,Humber Summit,M9L
9,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",M5V


<h2> print the number of rows of the cleaned dataframe

In [69]:
print(df_toronto_grouped.shape)

(103, 3)


<h2>geographical coordinates of each postal code

In [79]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
df_data.rename(columns={"Postal Code": "Postcode"}, inplace=True)
df_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2> Merging two tables

In [82]:
toronto_df_new = df_toronto_grouped.merge(df_data, on="Postcode", how="left")
toronto_df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [84]:
column_names = ["Postcode", "Borough", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["Postcode"]==postcode], ignore_index=True)
    
test_df

,Borough,Latitude,Longitude,Neighbourhood,Postcode
0,Downtown Toronto,43.657952,-79.387383,Central Bay Street,M5G
1,North York,43.803762,-79.363452,Hillcrest Village,M2H
2,East York,43.706397,-79.309937,"Woodbine Gardens, Parkview Hill",M4B
3,Scarborough,43.744734,-79.239476,Scarborough Village,M1J
4,East York,43.709060,-79.363452,Leaside,M4G
5,East Toronto,43.659526,-79.340923,Studio District,M4M
6,Scarborough,43.750072,-79.295849,"Maryvale, Wexford",M1R
7,Etobicoke,43.739416,-79.588437,"Albion Gardens, Beaumond Heights, Humbergate, ...",M9V
8,North York,43.756303,-79.565963,Humber Summit,M9L
9,Downtown Toronto,43.628947,-79.394420,"CN Tower, Bathurst Quay, Island airport, Harbo...",M5V
